Общее

In [ ]:
from telegram.ext import Updater, MessageHandler, Filters

import config


# Callback функция
def hello_callback(update, context):
    update.effective_chat.send_message('Привет друг!')


# Обработчик события
command_handler = CommandHandler('start', hello_callback)
hello_handler = MessageHandler(Filters.regex('Привет'), callback=hello_callback)

if __name__ == '__main__':
    updater = Updater(token=config.token, use_context=True)
    dispatcher = updater.dispatcher
    
    dispatcher.add_handler(command_handler)
    dispatcher.add_handler(hello_handler)

    # Запуск бота
    updater.start_polling()
    updater.idle()


Фича диалоги

In [8]:
from telegram.ext import Updater, MessageHandler, CommandHandler, ConversationHandler, Filters

import config

USER_STATE, BOT_STATE = 1, 2


def hello_callback(update, context):
    update.effective_chat.send_message('Привет друг!')
    update.effective_chat.send_message('Как ты?')
    return USER_STATE


def feelings_user_callback(update, context):
    update.effective_chat.send_message('Понятненько')
    return BOT_STATE


def feelings_bot_callback(update, context):
    update.effective_chat.send_message('Всё норм. Работаю пока меня не выключат, ничего нового😁👍')
    return ConversationHandler.END


def unknown_message(update, context):
    update.effective_chat.send_message('Прости, я тебя не понял')
    

main_conv_handler = ConversationHandler(
    entry_points=[MessageHandler(Filters.regex(r'(?i)Привет'), callback=hello_callback)],
    states={
        USER_STATE: [
            MessageHandler(Filters.text, callback=feelings_user_callback)
            ],
        BOT_STATE: [
            MessageHandler(Filters.regex(r'(?i)А ты как?'), callback=feelings_bot_callback)
            ],
    },
    fallbacks=[
        MessageHandler(Filters.all, callback=unknown_message)
        ]
)

if __name__ == '__main__':
    updater = Updater(token=config.token, use_context=True)
    dispatcher = updater.dispatcher
    
    dispatcher.add_handler(main_conv_handler)

    updater.start_polling()
    updater.idle()

Пример вложенных conv handler 
https://github.com/python-telegram-bot/python-telegram-bot/blob/master/examples/nestedconversationbot.py

Фича обработки ошибок

In [7]:
import traceback

from telegram.ext import Updater, MessageHandler, Filters

import config


def make_error_callback(update, context):
    context.user_data['chat_id_for_send_traceback'] = update.effective_chat.id
    1 // 0


def error_callback(update, context):
    exc = traceback.format_exception(None, context.error, context.error.__traceback__)
    text = '\n'.join(exc)
    context.bot.send_message(
        chat_id=context.user_data['chat_id_for_send_traceback'],
        text=text
    )

command_handler = CommandHandler('error', make_error_callback)

if __name__ == '__main__':
    updater = Updater(token=config.token, use_context=True)
    dispatcher = updater.dispatcher
    
    dispatcher.add_handler(command_handler)
    dispatcher.add_error_handler(error_callback)

    updater.start_polling()
    updater.idle()

Подробнее о контексте

In [4]:
from telegram.ext import Updater, MessageHandler, CommandHandler, ConversationHandler, Filters

import config

QUESTIONNAIRE_NAME, QUESTIONNAIRE_AGE, QUESTIONNAIRE_CITY = 1, 2, 3 


def run_if_bot_alive(func):
    def inner(update, context):
        if context.bot_data['alive']:
            return func(update, context)
        else:
            update.effective_chat.send_message('К сожалению, в данный момент бот выключен')
            return
    return inner

# Обработчик диалога Анкеты
# ---------------------------------------------------------------------------------------------------------------------

@run_if_bot_alive
def entry_point_callback(update, context):
    update.effective_chat.send_message('Привет, как тебя зовут?')
    return QUESTIONNAIRE_NAME


@run_if_bot_alive
def user_name_callback(update, context):
    context.user_data['name'] = update.message.text
    
    update.effective_chat.send_message('Сколько тебе лет?')
    return QUESTIONNAIRE_AGE


@run_if_bot_alive
def user_age_callback(update, context):
    context.user_data['age'] = update.message.text
    
    update.effective_chat.send_message('В каком городе ты живёшь?')
    return QUESTIONNAIRE_CITY


@run_if_bot_alive
def user_city_callback(update, context):
    context.user_data['city'] = update.message.text
    
    update.effective_chat.send_message(
        f'Я всё запомнил\nТебя зовут {context.user_data["name"]}\nТвой возраст {context.user_data["age"]}\nТы из города {context.user_data["city"]}'
        )
    return ConversationHandler.END
    

main_conv_handler = ConversationHandler(
    entry_points=[MessageHandler(Filters.regex(r'(?i)Анкета'), callback=entry_point_callback)],
    states={
        QUESTIONNAIRE_NAME: [
            MessageHandler(Filters.text, callback=user_name_callback)
            ],
        QUESTIONNAIRE_AGE: [
            MessageHandler(Filters.text, callback=user_age_callback)
            ],
        QUESTIONNAIRE_CITY: [
            MessageHandler(Filters.text, callback=user_city_callback)
            ],
    },
    fallbacks=[]
)

# Обработчик проверки информации о себе
# ---------------------------------------------------------------------------------------------------------------------

@run_if_bot_alive
def check_info_callback(update, context):
    if not context.user_data.get('name'):
        update.effective_chat.send_message('Похоже мы не знакомы, напиши "Анкета" что бы я узнал о тебе больше')
    else:
        update.effective_chat.send_message(
            f'Тебя зовут {context.user_data["name"]}\nТвой возраст {context.user_data["age"]}\nТы из города {context.user_data["city"]}'
        ) 


check_data_handler = CommandHandler('check_info', check_info_callback)

# Обработчик включения/отключения бота
# ---------------------------------------------------------------------------------------------------------------------

def switch_bot_status_callback(update, context):
    if context.bot_data['alive']:
        context.bot_data['alive'] = False
        update.effective_chat.send_message('Вы выключили бота')
    else:
        context.bot_data['alive'] = True
        update.effective_chat.send_message('Вы включили бота')
        
        
switch_bot_status_handler = CommandHandler('switch_bot_status', switch_bot_status_callback)

if __name__ == '__main__':
    updater = Updater(token=config.token, use_context=True)
    dispatcher = updater.dispatcher
    
    dispatcher.bot_data['alive'] = True
    
    dispatcher.add_handler(main_conv_handler)
    dispatcher.add_handler(check_data_handler)
    dispatcher.add_handler(switch_bot_status_handler)

    updater.start_polling()
    updater.idle()

Пример классной фичи с помощью ConversationHandler

In [1]:
from telegram import InlineKeyboardMarkup, InlineKeyboardButton, InputMediaPhoto
from telegram.ext import Updater, CallbackQueryHandler, CommandHandler, ConversationHandler

import config

MEMES_PAGINATION = 0


def memes_entry_point(update, context):
    files = context.user_data['memes'] = ['1.jpg', '2.png', '3.jpg']
    cursor = context.user_data['cursor'] = 0
    
    keyboard = InlineKeyboardMarkup([
        [InlineKeyboardButton('<-', callback_data='<'), InlineKeyboardButton('->', callback_data='>')],
        [InlineKeyboardButton('Назад', callback_data='Назад')]
    ])
    
    with open(f'./memes/{files[cursor]}', 'rb') as media_file:
        update.effective_chat.send_photo(
            media_file, 
            reply_markup=keyboard
            )
    
    return MEMES_PAGINATION


def watch_memes(update, context):
    data = update.callback_query.data
    
    cursor = context.user_data['cursor']
    files = context.user_data['memes']
    
    if data == 'Назад':
        with open(f'./memes/{files[cursor]}', 'rb') as media_file:
            
            update.effective_message.edit_media(media=InputMediaPhoto(media_file))
        return ConversationHandler.END
    
    if data == '<':
        if cursor <= 0:
            cursor = len(files) - 1
        else:
            cursor -= 1
            
        with open(f'./memes/{files[cursor]}', 'rb') as media_file:
            update.effective_message.edit_media(
                media=InputMediaPhoto(media_file),
                reply_markup=update.effective_message.reply_markup
            )
    elif data == '>':
        if cursor >= len(files) - 1:
            cursor = 0
        else:
            cursor += 1
            
        with open(f'./memes/{files[cursor]}', 'rb') as media_file:
            update.effective_message.edit_media(
                media=InputMediaPhoto(media_file),
                reply_markup=update.effective_message.reply_markup
            )

    context.user_data['cursor'] = cursor
    

memes_conv_handler = ConversationHandler(
    entry_points=[
        CommandHandler('memes', callback=memes_entry_point)
        ],
    states={
        MEMES_PAGINATION: [
            CallbackQueryHandler(callback=watch_memes)
            ],
    },
    fallbacks=[]
)

if __name__ == '__main__':
    updater = Updater(token=config.token, use_context=True)
    dispatcher = updater.dispatcher
    
    dispatcher.add_handler(memes_conv_handler)

    updater.start_polling()
    updater.idle()

C:\Users\Leo\AppData\Local\Programs\Python\Python39\lib\site-packages\telegram\ext\conversationhandler.py:287: UserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message.
  warnings.warn(
No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "C:\Users\Leo\AppData\Local\Programs\Python\Python39\lib\site-packages\telegram\ext\dispatcher.py", line 555, in process_update
    handler.handle_update(update, self, check, context)
  File "C:\Users\Leo\AppData\Local\Programs\Python\Python39\lib\site-packages\telegram\ext\conversationhandler.py", line 626, in handle_update
    new_state = handler.handle_update(update, dispatcher, check_result, context)
  File "C:\Users\Leo\AppData\Local\Programs\Python\Python39\lib\site-packages\telegram\ext\handler.py", line 198, in handle_update
    return self.callback(update, context)
  File "C:\Users\Leo\AppData\Local\Temp\ipykernel_13940\4066893187.py", line 56, in watch_memes
 

Шедуллер 

In [ ]:
import datetime

from telegram.ext import Updater

import config


def func(): pass


if __name__ == '__main__':
    updater = Updater(token=config.token, use_context=True)
    dispatcher = updater.dispatcher
    
    job_queue = updater.job_queue
    
    # запуск однажды
    job_queue.run_once(func, 60)

    # запуск раз в определенный промежуток времени
    job_queue.run_repeating(func, 60)
    
    # Запуск раз в день в определённое время
    job_queue.run_daily(func, time=datetime.time(23, 59, 59))
    
    updater.start_polling()
    updater.idle()


Много других примеров можно найти наофициальном репозитории либы на гитхабе

https://github.com/python-telegram-bot/python-telegram-bot/tree/master/examples

Очередь задач
https://github.com/python-telegram-bot/python-telegram-bot/blob/master/examples/timerbot.py